In [2]:
import yaml
import os
config_files = [
    "n_warehouse_21_5_real_lost_demand", 
    "n_warehouse_21_4_real_lost_demand", 
    "n_warehouse_21_3_real_lost_demand", 
    "n_warehouse_21_2_real_lost_demand"
]

for config_file in config_files:
    with open(f'{config_file}.yml', 'r') as file:
        config = yaml.safe_load(file)
    n_stores = config['problem_params']['n_stores']
    n_warehouses = config['problem_params']['n_warehouses']
    for store_idx in range(n_stores):
        for warehouse_idx in range(n_warehouses):
            config_copy = yaml.safe_load(yaml.dump(config))
            config_copy['problem_params']['n_stores'] = 1
            config_copy['problem_params']['n_warehouses'] = 1
            
            # save and load
            config_copy['store_params']['demand']['file_location'] = f'/user/ml4723/Prj/NIC/data_files/favorita_{n_stores}_stores/separate/weekly_sales_{store_idx}.pt'
            config_copy['store_params']['underage_cost']['file_location'] = f'/user/ml4723/Prj/NIC/data_files/favorita_{n_stores}_stores/separate/underage_costs_{store_idx}.pt'
            config_copy['store_params']['holding_cost']['file_location'] = f'/user/ml4723/Prj/NIC/data_files/favorita_{n_stores}_stores/separate/holding_costs_{store_idx}.pt'


            config_copy['store_params']['holding_cost']['sample_across_instances'] = False
            config_copy['store_params']['holding_cost']['vary_across_samples'] = False
            config_copy['store_params']['underage_cost']['sample_across_instances'] = False
            config_copy['store_params']['underage_cost']['vary_across_samples'] = False

            # Update warehouse specific parameters
            config_copy['warehouse_params']['lead_time']['value'] = config['warehouse_params']['lead_time']['value'][warehouse_idx]
            config_copy['warehouse_params']['holding_cost']['value'] = config['warehouse_params']['holding_cost']['value'][warehouse_idx]
            config_copy['warehouse_params']['edge_initial_cost']['value'] = config['warehouse_params']['edge_initial_cost']['value'][warehouse_idx]
            
            if config['warehouse_params']['edges']['value'][warehouse_idx][store_idx] == 0:
                continue
            config_copy['warehouse_params']['edge_lead_times']['value'] = [[config['warehouse_params']['edge_lead_times']['value'][warehouse_idx][store_idx]]]
            config_copy['warehouse_params']['edges']['value'] = [[config['warehouse_params']['edges']['value'][warehouse_idx][store_idx]]]
            
            # Save config
            output_dir = f'/user/ml4723/Prj/NIC/config_files/settings/separate/{config_file}'
            os.makedirs(output_dir, exist_ok=True)
            with open(f'{output_dir}/{store_idx}_{warehouse_idx}.yml', 'w') as file:
                yaml.dump(config_copy, file)


In [6]:
import torch
import os

# Load the full tensors
underage_costs = torch.load('/user/ml4723/Prj/NIC/data_files/favorita_21_stores/underage_costs.pt')
holding_costs = torch.load('/user/ml4723/Prj/NIC/data_files/favorita_21_stores/holding_costs.pt')

# Create output directory if it doesn't exist
output_dir = '/user/ml4723/Prj/NIC/data_files/favorita_21_stores/separate'
os.makedirs(output_dir, exist_ok=True)

# Split and save individual store data
for store_idx in range(21):
    # Split underage costs
    store_underage = underage_costs[:, store_idx:store_idx+1]
    torch.save(store_underage, os.path.join(output_dir, f'underage_costs_{store_idx}.pt'))
    
    # Split holding costs 
    store_holding = holding_costs[:, store_idx:store_idx+1]
    torch.save(store_holding, os.path.join(output_dir, f'holding_costs_{store_idx}.pt'))

In [7]:
import torch
import os

# Load the full tensor
data = torch.load('/user/ml4723/Prj/NIC/data_files/favorita_21_stores/weekly_sales.pt')

# Create output directory if it doesn't exist
output_dir = '/user/ml4723/Prj/NIC/data_files/favorita_21_stores/separate'
os.makedirs(output_dir, exist_ok=True)

# Split and save individual store data
for store_idx in range(21):
    store_data = data[:, store_idx:store_idx+1, :]  # Shape: (288, 1, ?)
    output_path = os.path.join(output_dir, f'weekly_sales_{store_idx}.pt')
    torch.save(store_data, output_path)

In [5]:
%load_ext autoreload
%autoreload 2

import analysis.ray_results_interpreter as rri
import glob
import os
import yaml

config_files = [
    "n_warehouse_21_5_real_lost_demand", 
    "n_warehouse_21_4_real_lost_demand", 
    "n_warehouse_21_3_real_lost_demand", 
    "n_warehouse_21_2_real_lost_demand"
]

results_interpretor = rri.RayResultsinterpreter()
def custom_data_filler(out_row, reference_row):
    out_row['path'] = reference_row['path']

def default_condition_setter(condition_name):
    return None

sort_by = 'dev_loss'
pick_row_from_run_by = 'dev_loss'

for config_file in config_files:
    print(f"\nProcessing config: {config_file}")
    with open(f'{config_file}.yml', 'r') as file:
        config = yaml.safe_load(file)
        
    # Get the appropriate yml files for this config
    base_path = f'/user/ml4723/Prj/NIC/config_files/settings/separate/{config_file}'
    yml_files = []
    if os.path.isdir(base_path):
        for yml_file in glob.glob(os.path.join(base_path, '*.yml')):
            yml_files.append(yml_file.split('settings/')[1].replace('.yml', ''))
    
    conditions = {
        'config': yml_files,
    }

    df = results_interpretor.make_table({1: "/user/ml4723/Prj/NIC/ray_results/separate_store/data_driven_net_n_warehouses_real"},
        conditions,
        default_condition_setter,
        custom_data_filler,
        sort_by=sort_by,
        pick_row_from_run_by=pick_row_from_run_by,
        test_loss_limit=100)

    # Extract store and warehouse indices from config column
    df['store_index'] = df['config'].str.split('/').str[-1].str.split('_').str[0].astype(int)
    df['warehouse_index'] = df['config'].str.split('/').str[-1].str.split('_').str[1].astype(int)

    # Group by store_index and find warehouse with lowest dev loss for each store
    best_warehouses = df.groupby('store_index').apply(lambda x: x.loc[x['Test Loss'].idxmin()][['store_index', 'warehouse_index', 'Test Loss']])
    # print("\nBest warehouse assignments by dev loss:")
    # print(best_warehouses)
    
    # Get best warehouse assignments from best_warehouses DataFrame
    best_assignments = {}
    for _, row in best_warehouses.iterrows():
        store = row['store_index'] 
        warehouse = row['warehouse_index']
        best_assignments[store] = warehouse

    # Create new edge matrix with only best warehouse connections
    n_warehouses = int(config_file.split('_')[3])
    n_stores = 21
    new_edges = [[0]*n_stores for _ in range(n_warehouses)]

    for store in range(n_stores):
        if store in best_assignments:
            best_warehouse = int(best_assignments[store])  # Convert float to int
            new_edges[best_warehouse][store] = 1
    
    print("  'edges':")
    print("    'value': [")
    for row in new_edges:
        print("             ", row, ",") 
    print("            ]")

    # Load the configuration data
    edges = config['warehouse_params']['edges']['value']
    edge_lead_times = config['warehouse_params']['edge_lead_times']['value']
    edge_initial_costs = config['warehouse_params']['edge_initial_cost']['value']
    
    # Create new edge matrix with only lowest cost warehouse connections
    n_warehouses = len(edges)
    n_stores = len(edges[0])
    min_cost_edges = [[0]*n_stores for _ in range(n_warehouses)]
    
    # For each store, find the warehouse with lowest edge_initial_cost
    for store in range(n_stores):
        min_cost = float('inf')
        min_cost_warehouse = -1
        
        for warehouse in range(n_warehouses):
            # Check if there's a connection and if the cost is lower
            if edges[warehouse][store] == 1 and edge_initial_costs[warehouse] < min_cost:
                min_cost = edge_initial_costs[warehouse]
                min_cost_warehouse = warehouse
        
        # Set the connection for the lowest cost warehouse
        if min_cost_warehouse != -1:
            min_cost_edges[min_cost_warehouse][store] = 1
    
    print("  'edges_with_min_cost':")
    print("    'value': [")
    for row in min_cost_edges:
        print("             ", row, ",") 
    print("            ]")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Processing config: n_warehouse_21_5_real_lost_demand


/tmp/ipykernel_17346/2922734437.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_warehouses = df.groupby('store_index').apply(lambda x: x.loc[x['Test Loss'].idxmin()][['store_index', 'warehouse_index', 'Test Loss']])


  'edges':
    'value': [
              [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1] ,
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ,
              [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0] ,
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] ,
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ,
            ]
  'edges_with_min_cost':
    'value': [
              [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1] ,
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ,
              [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0] ,
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ,
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] ,
            ]

Processing config: n_warehouse_21_4_real_lost_demand


/tmp/ipykernel_17346/2922734437.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_warehouses = df.groupby('store_index').apply(lambda x: x.loc[x['Test Loss'].idxmin()][['store_index', 'warehouse_index', 'Test Loss']])


  'edges':
    'value': [
              [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1] ,
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ,
              [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0] ,
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] ,
            ]
  'edges_with_min_cost':
    'value': [
              [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1] ,
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ,
              [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0] ,
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] ,
            ]

Processing config: n_warehouse_21_3_real_lost_demand


/tmp/ipykernel_17346/2922734437.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_warehouses = df.groupby('store_index').apply(lambda x: x.loc[x['Test Loss'].idxmin()][['store_index', 'warehouse_index', 'Test Loss']])


  'edges':
    'value': [
              [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1] ,
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ,
              [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0] ,
            ]
  'edges_with_min_cost':
    'value': [
              [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1] ,
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ,
              [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0] ,
            ]

Processing config: n_warehouse_21_2_real_lost_demand
  'edges':
    'value': [
              [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1] ,
              [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0] ,
            ]
  'edges_with_min_cost':
    'value': [
              [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1] ,
              [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0

/tmp/ipykernel_17346/2922734437.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_warehouses = df.groupby('store_index').apply(lambda x: x.loc[x['Test Loss'].idxmin()][['store_index', 'warehouse_index', 'Test Loss']])
